# Running time code

### **This notebook shows how long the source code takes to get the final output. Here, we use all functions in *Gravmag* and also *Data filtering*.**

In [2]:
# Copying all usefull libraries
import numpy as np
import gravmag as gmc
import datafiltering as df
import time

### Define all discription for running the codes, like number of points and models.

In [3]:
# Definition for number of points and grid creation
n = 500
x = np.linspace(-1000., 1000., n)
y = x
z = np.random.normal(loc = -50., scale = 1., size = n)
# Creat the grid file
X, Y = np.meshgrid(x, y)

### Define both synthetic models: prism and sphere.

In [4]:
spheregrav = (0., 0., 500., 500., 2673.) 
spheremag = (0., 0., 500., 500., 2.25) 
prismgrav = (-250., 250., -250., 250., 100., 300., 2673.)
prismmag = (-250., 250., -250., 250., 100., 300., 2.250)

**(1) Gravity sources**

In [5]:
print 'By using a rectangular prism:'
print 'Gravitational potential'
%timeit -n 10 gmc.potential(X, Y, z, prismgrav)
print 'Gx component:'
%timeit -n 10 gmc.prism_gx(X, Y, z, prismgrav)
print 'Gy component:'
%timeit -n 10 gmc.prism_gy(X, Y, z, prismgrav)
print 'Gz component:'
%timeit -n 10 gmc.prism_gz(X, Y, z, prismgrav)
print
print 'By using a solid sphere:'
print 'Gz component:'
%timeit -n 10 gmc.potential(X, Y, z, prismgrav)

By using a rectangular prism:
Gravitational potential
10 loops, best of 3: 844 ms per loop
Gx component:
10 loops, best of 3: 494 ms per loop
Gy component:
10 loops, best of 3: 625 ms per loop
Gz component:
10 loops, best of 3: 604 ms per loop

By using a solid sphere:
Gz component:
10 loops, best of 3: 1.27 s per loop


**(2) Magnetic sources**

In [6]:
print 'By using a rectangular prism:'
print 'Total field anomaly'
%timeit -n 10 gmc.prism_tf(X, Y, z, prismmag, (42., 22.), (42., 22.))
print
print 'By using a solid sphere:'
print 'Bx component:'
%timeit -n 10 gmc.sphere_bx(X, Y, z, spheremag, (42., 22.))
print 'By component:'
%timeit -n 10 gmc.sphere_by(X, Y, z, spheremag, (42., 22.))
print 'Bz component:'
%timeit -n 10 gmc.sphere_bz(X, Y, z, spheremag, (42., 22.))
print 'True total field anomaly:'
%timeit -n 10 gmc.sphere_tf(X, Y, z, spheremag, (42., 22.), (42., 22.), 23500.)
print 'Approximated total field anomaly:'
%timeit -n 10 gmc.sphere_tf(X, Y, z, spheremag, (42., 22.), (42., 22.), 23500.)

By using a rectangular prism:
Total field anomaly
10 loops, best of 3: 1.22 s per loop

By using a solid sphere:
Bx component:
10 loops, best of 3: 62.7 ms per loop
By component:
10 loops, best of 3: 72.5 ms per loop
Bz component:
10 loops, best of 3: 73.3 ms per loop
True total field anomaly:
10 loops, best of 3: 243 ms per loop
Approximated total field anomaly:
10 loops, best of 3: 217 ms per loop


** (3) Transformation and filters for potential data**

In [7]:
# Compute the simple gravity and magnetic anomalies
datagrav = gmc.prism_gz(X, Y, z, prismgrav)
datamag = gmc.prism_tf(X, Y, z, prismmag, (42., 22.), (42., 22.))

* Upward continuation

In [11]:
print 'Upward continuation for gravity data:'
%timeit -n 10 df.continuation(X, Y, datagrav, 500.)
print 'Upward continuation for magnetic data:'
%timeit -n 10 df.continuation(X, Y, datamag, 500.)

Upward continuation for gravity data:
10 loops, best of 3: 89.7 ms per loop
Upward continuation for magnetic data:
10 loops, best of 3: 103 ms per loop


* Reduction filter

In [12]:
print 'Reduction to magnetic Pole:'
%timeit -n 10 df.reduction(X, Y, datamag, (42., 22.), (42., 22.), (90., 0.), (90., 0.))
print 'Reduction to magnetic Equator:'
%timeit -n 10 df.reduction(X, Y, datamag, (42., 22.), (42., 22.), (45., 0.), (45., 0.))

Reduction to magnetic Pole:
10 loops, best of 3: 197 ms per loop
Reduction to magnetic Equator:
10 loops, best of 3: 197 ms per loop


* Derivatives

In [18]:
print 'X derivative for gravity data:'
%timeit -n 10 df.xderiv(x, y, datagrav, 1)
print 'Y derivative for gravity data:'
%timeit -n 10 df.yderiv(x, y, datagrav, 1)
print 'Z derivative for gravity data:'
%timeit -n 10 df.zderiv(x, y, datagrav, 1)
print '----------------------------------------'
print 'X derivative for magnetic data:'
%timeit -n 10 df.xderiv(x, y, datamag, 1)
print 'Y derivative for magnetic data:'
%timeit -n 10 df.yderiv(x, y, datamag, 1)
print 'Z derivative for magnetic data:'
%timeit -n 10 df.zderiv(x, y, datamag, 1)

X derivative for gravity data:
10 loops, best of 3: 76.8 ms per loop
Y derivative for gravity data:
10 loops, best of 3: 81.5 ms per loop
Z derivative for gravity data:
10 loops, best of 3: 82.7 ms per loop
----------------------------------------
X derivative for magnetic data:
10 loops, best of 3: 83.5 ms per loop
Y derivative for magnetic data:
10 loops, best of 3: 83.5 ms per loop
Z derivative for magnetic data:
10 loops, best of 3: 91.6 ms per loop


* Transformations

In [19]:
print 'Total gradient for magnetic data:'
%timeit -n 10 df.totalgrad(x, y, datamag)
print 'Total gradient for gravity data:'
%timeit -n 10 df.totalgrad(x, y, datagrav)
print '-------------------------------------'
print 'Tilt angle for magnetic data:'
%timeit -n 10 df.tilt(x, y, datamag)
print '-------------------------------------'
print 'Tilt angle for magnetic data:'
%timeit -n 10 df.pseudograv(x, y, datamag, (42., 22.), (42., 22.), 2673., 2.25)

Total gradient for magnetic data:
10 loops, best of 3: 263 ms per loop
Total gradient for gravity data:
10 loops, best of 3: 253 ms per loop
-------------------------------------
Tilt angle for magnetic data:
10 loops, best of 3: 279 ms per loop
-------------------------------------
Tilt angle for magnetic data:
10 loops, best of 3: 155 ms per loop


* Cross correlation coefficient

In [23]:
print 'Cross correlation:'
%timeit -n 100 df.cccoef(datamag, datagrav)

Cross correlation:
100 loops, best of 3: 23.6 ms per loop
